# Strands를 사용한 단백질 설계 에이전트

이 노트북에서는 오픈 소스 Strands Agents 프레임워크를 사용하여 단백질 설계 에이전트를 만들어보겠습니다.

## 학습 목표 

- Strands 프레임워크를 사용한 단백질 설계 에이전트 구축
- AWS HealthOmics 워크플로우를 실행하는 방법
- 워크플로우 모니터링 및 결과 분석


## 1. 환경 설정

#### Strands 에이전트 및 필수 종속성 설치

In [ ]:
%pip install strands-agents strands-agents-tools --quiet

#### 아래에 표시된 최신 버전의 boto3 확인
아래에 출력된 boto3 버전이 **1.37.1** 이상인지 확인하세요.

In [ ]:
%pip show boto3

#### 필수 라이브러리 가져오기

In [ ]:
import boto3
import json
import time
import uuid
import os
from strands import Agent, tool
from strands.models import BedrockModel
from urllib.parse import urlparse

# AWS 계정 정보 가져오기
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']



In [ ]:
session = boto3.Session(
    #aws_access_key_id='your_access_key',
    #aws_secret_access_key='your_secret_key',
    #aws_session_token='your_session_token',  # If using temporary credentials
    #region_name='us-west-2',
    #profile_name='your-profile'  # Optional: Use a specific profile
)
region = session.region_name

### 전제 조건

[00-setup_environment.ipynb](00-setup_environment.ipynb)에서 노트북 환경 설정을 실행하세요.

또한 `stacks/protein_design_stack.yaml`에서 단백질 설계 CloudFormation 스택을 배포했는지 확인하세요.

<img src="https://github.com/hsr87/strands-agents-for-life-science/blob/main/notebook/images/protein-stack-completion.png?raw=true" width="400" alt="CloudFormation Checking">


### AWS 클라이언트 및 구성 설정
도구에서 사용할 AWS 서비스에 대한 클라이언트를 정의합니다.

In [ ]:
STACK_NAME = 'protein-design-stack'  # 여러분의 CloudFormation 스택 이름
DEFAULT_WORKFLOW_ID = None  # 스택 출력에서 검색됩니다
DEFAULT_ROLE_ARN = None     # 스택 출력에서 검색됩니다
DEFAULT_S3_BUCKET = f"{STACK_NAME}-{account_id}-{region}"    # CloudFormation 에 사전 정의된 형식으로 정해짐.

# 도구에서 사용할 환경 변수 설정
os.environ['STACK_NAME'] = STACK_NAME

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Stack Name: {STACK_NAME}")

### CloudFormation 스택 출력 가져오기

In [ ]:
# CloudFormation 스택 출력 가져오기
cf_client = boto3.client('cloudformation')

try:
    response = cf_client.describe_stacks(StackName=STACK_NAME)
    stack = response['Stacks'][0]

    # 출력에서 가져오기
    outputs = stack.get('Outputs', [])
    for output in outputs:
        key = output['OutputKey']
        value = output['OutputValue']
        if key == 'WorkflowId':
            DEFAULT_WORKFLOW_ID = value
        elif key == 'WorkflowExecutionRoleArn':
            DEFAULT_ROLE_ARN = value
            
    parameters = stack.get('Parameters', [])

    
    print(f"Workflow ID: {DEFAULT_WORKFLOW_ID}")
    print(f"Role ARN: {DEFAULT_ROLE_ARN}")
    print(f"S3 Bucket: {DEFAULT_S3_BUCKET}")
    
    # 스택 정보로 도구 구성
    from utils.protein_design_tools import set_stack_config
    set_stack_config(
        stack_name=STACK_NAME,
        workflow_id=DEFAULT_WORKFLOW_ID,
        role_arn=DEFAULT_ROLE_ARN,
        s3_bucket=DEFAULT_S3_BUCKET
    )
    
except Exception as e:
    print(f"스택 출력 가져오기 오류: {e}")
    print("구성 변수를 수동으로 업데이트해 주세요")

## 2. Strands 에이전트 생성
이 섹션에서는 Strands 프레임워크를 사용하여 에이전트를 생성합니다

### 에이전트 구성 및 지침 정의

In [ ]:
protein_agent_name = 'Protein-Design-Agent-Strands'
protein_agent_description = "Strands 프레임워크를 사용한 AWS HealthOmics 워크플로우 기반 단백질 설계 및 최적화 에이전트"
protein_agent_instruction = """
당신은 방향성 진화 알고리즘을 사용하여 연구자들이 단백질 서열을 최적화하는 데 도움을 주는 단백질 설계 전문 AI입니다.
AWS HealthOmics 워크플로우를 트리거하여 단백질 서열 최적화를 수행하고 진행 상황을 모니터링할 수 있습니다.

당신의 기능은 다음과 같습니다:
1. 사용자 정의 매개변수로 단백질 설계 최적화 워크플로우 시작
2. 실행 중인 워크플로우의 상태 모니터링
3. 완료된 최적화에서 결과 검색 및 분석

단백질 서열 작업 시:
- 서열이 유효한 아미노산 문자만 포함하는지 검증
- 최적화 과정에 대한 명확한 설명 제공
- 사용자가 결과와 그 의미를 이해할 수 있도록 도움

항상 도움이 되고 단백질 설계 과정에 대한 자세한 정보를 제공하세요.
"""

#### Strands 에이전트용 도구 정의

이 도구들은 단백질 설계 작업을 위해 AWS HealthOmics 워크플로우를 호출 및 모니터링 하게 될 것 입니다.

In [ ]:
# Bedrock 에이전트 액션 그룹과 일치하는 업데이트된 도구 가져오기
from utils.protein_design_tools import trigger_aho_workflow, monitor_aho_workflow

print("도구가 성공적으로 가져와졌습니다:")
print(f"- trigger_aho_workflow: {trigger_aho_workflow.__doc__.split('Args:')[0].strip()}")
print(f"- monitor_aho_workflow: {monitor_aho_workflow.__doc__.split('Args:')[0].strip()}")

#### Strands 에이전트 생성

In [ ]:
# Bedrock 모델 생성
model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-5-20250929-v1:0",
    boto_session=session
)

protein_agent = Agent(
    system_prompt=protein_agent_instruction,
    model=model,
    tools=[trigger_aho_workflow, monitor_aho_workflow]
)

print(f"Strands 프레임워크로 단백질 설계 에이전트를 생성했습니다")

# 에이전트 테스트
몇 가지 예제 쿼리로 단백질 설계 에이전트를 테스트해보겠습니다.

#### (옵션) 디버깅 

StrandsAgents 의 로깅 모듈을 활용해 필요하다면 디버깅할 수 있습니다.

In [ ]:
#https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/logs/
#import logging

# 루트 strands 로거 구성
#logging.getLogger("strands").setLevel(logging.DEBUG)

# 로그를 보기 위한 핸들러 추가
#logging.basicConfig(
#    format="%(levelname)s | %(name)s | %(message)s", 
#    handlers=[logging.StreamHandler()]
#)

#### 테스트 1: 단백질 최적화 시작

In [ ]:
# 단백질 최적화 시작 테스트
test_sequence = "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS"

query = f"이 단백질 서열을 최적화해 주세요: {test_sequence}"

print("쿼리:", query)

try:
    # 에이전트 실행
    print("\n응답:")
    protein_agent(query)
except Exception as e:
    print(f"에이전트 실행 중 오류: {e}")
    import traceback
    traceback.print_exc()

#### 테스트 2: 워크플로우 상태 모니터링

아래와 같이 HealthOmics 콘솔로 진입하여 Run 메뉴를 통해 앞에서 실행한 Run ID에 대한 상황을 조회할 수도 있습니다. (여기 예시 그림에서는 Run ID의 경우 2951139 이며 사용자마다 이 값은 다를 것입니다)

<img src="https://github.com/hsr87/strands-agents-for-life-science/blob/main/notebook/images/healthomics.png?raw=true" width="400" alt="HealthOmics Run">


In [ ]:
# 워크플로우 모니터링 테스트 (이전 테스트의 실행 ID 사용)
# 'YOUR_RUN_ID'를 이전 테스트의 실제 실행 ID로 교체하세요
test_run_id = "{test_run_id}"  # 실제 실행 ID로 업데이트하세요

query = f"워크플로우 실행 {test_run_id}의 상태를 확인해 주세요"

print("쿼리:", query)
print("\n응답:")
response = protein_agent(query)

#### 테스트 3: 사용자 정의 매개변수를 사용한 고급 최적화

In [ ]:
# 사용자 정의 매개변수로 테스트
query = "ACDEFGHIKLMNPQRSTVWY 서열에 대해 20개의 병렬 체인과 200단계로 단백질 최적화를 실행해 주세요"

print("쿼리:", query)
print("\n응답:")
response = protein_agent(query)

In [ ]:
# 메트릭: https://strandsagents.com/latest/documentation/docs/user-guide/observability-evaluation/metrics/

# AgentResult를 통해 메트릭에 액세스
print(f"총 토큰: {response.metrics.accumulated_usage['totalTokens']}")
print(f"실행 시간: {sum(response.metrics.cycle_durations):.2f}초")
print(f"사용된 도구: {list(response.metrics.tool_metrics.keys())}")

## 결론

이 노트북에서는 Strands Agents 프레임워크를 사용하여 AWS HealthOmics 워크플로우 기반의 단백질 설계 에이전트를 성공적으로 구현했습니다.

### 주요 구현 내용:

#### 1. 단백질 설계 에이전트 구축
- **Strands 프레임워크**: 오픈소스 에이전트 프레임워크를 활용한 단백질 설계 전문 AI 구현
- **AWS HealthOmics 연동**: 클라우드 기반 생명과학 워크플로우 실행 및 관리

#### 2. 핵심 도구 구현
- **trigger_aho_workflow**: 사용자 정의 매개변수로 단백질 최적화 워크플로우 시작
- **monitor_aho_workflow**: 실행 중인 워크플로우의 상태 실시간 모니터링
- **CloudFormation 통합**: 인프라 자동화를 통한 일관된 환경 구성

#### 3. 에이전트 기능
- 아미노산 서열 유효성 검증
- 사용자 정의 최적화 매개변수 설정 (병렬 체인 수, 최적화 단계 등)
- 워크플로우 실행 상태 추적 및 결과 분석
- 단백질 설계 과정에 대한 상세한 설명 제공

#### 4. AWS 서비스 통합
- **AWS HealthOmics**: 생명과학 워크플로우 실행 플랫폼
- **Amazon Bedrock**: Claude 4.5 Sonnet 모델을 통한 자연어 처리
- **Amazon S3**: 워크플로우 입출력 데이터 저장
- **AWS IAM**: 보안 역할 및 권한 관리

### 실제 활용 예시:

**단백질 최적화 워크플로우:**

사용자 입력: "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVSVIYSDGRRTFYGDSVKGRFTISRDTSTNTVYLQMNSLRVEDTAVYYCAKGRAAGTFDSWGQGTLVTVSS 서열을 최적화해 주세요"
↓ 에이전트 처리
• 서열 유효성 검증
• HealthOmics 워크플로우 트리거
• 실행 상태 모니터링
• 결과 분석 및 해석

**고급 매개변수 설정:**

"20개의 병렬 체인과 200단계로 단백질 최적화를 실행해 주세요"
→ 사용자 정의 매개변수로 최적화 성능 조정

### 기술적 특징:

#### 1. 확장성
- 클라우드에서 대규모 단백질 최적화 작업 처리
- 병렬 처리를 통한 효율적인 계산 자원 활용 (관리형 서비스 AWS HealthOmics 사용)

#### 2. 사용성
- 자연어 인터페이스로 복잡한 생명과학 워크플로우 간편 실행
- 실시간 모니터링을 통한 투명한 프로세스 추적

#### 3. 통합성
- AWS 생태계와의 완전한 통합
- CloudFormation을 통한 인프라 코드화

### 활용 분야:
- **신약 개발**: 치료용 단백질 최적화
- **효소 공학**: 산업용 효소 성능 향상
- **항체 설계**: 특이성과 친화도 개선
- **백신 개발**: 면역원성 최적화

이 구현을 통해 생명과학 연구자들은 복잡한 계산 생물학 지식 없이도 AI 에이전트와의 자연어 대화만으로 고급 단백질 설계 작업을 수행할 수 있게 되었습니다. **Strands Agents 프레임워크**와 **AWS HealthOmics**의 결합이 단백질 설계 분야의 접근성을 크게 향상시켰습니다.
